In [3]:
import os
import pandas as pd
import re

DATA_PATH = os.path.join('asum', 'out')
FILENAME = 'STO2-T7-S2(2)-A0.1-B0.001,0.1,0.1-G1.0,1.0-I1000-ProbWords.csv'

In [16]:
# Parse filename
n_topics = int(FILENAME.split('-')[1][1:])
n_sentiment = int(FILENAME.split('-')[2][1:].split('(')[-1][:-1])

print(f"Number of topics: {str(n_topics)}")
print(f"Number of sentiment: {str(n_sentiment)}")

Number of topics: 7
Number of sentiment: 2


In [22]:
df = pd.read_csv(os.path.join(DATA_PATH, FILENAME))
df = df.iloc[:,:n_topics*n_sentiment]

pos_sentiment = df.iloc[:,:n_topics]
neg_sentiment = df.iloc[:,n_topics:]

df.head()

,S0-T0,S0-T1,S0-T2,S0-T3,S0-T4,S0-T5,S0-T6,S1-T0,S1-T1,S1-T2,S1-T3,S1-T4,S1-T5,S1-T6
0,call (0.016),benghazi (0.176),sent (0.046),state (0.133),obama (0.007),agricultur (0.031),h (0.041),pm (0.108),state (0.009),assist (0.049),http (0.034),receiv (0.024),parti (0.020),percent (0.077)
1,work (0.011),comm (0.174),thank (0.039),date (0.070),would (0.007),develop (0.018),sid (0.031),secretari (0.054),would (0.006),secretari (0.040),time (0.027),percent (0.023),tori (0.014),thx (0.052)
2,get (0.011),hous (0.174),blackberri (0.035),dept (0.070),presid (0.006),approach (0.017),foreign (0.026),offic (0.053),govern (0.005),state (0.037),spoke (0.019),among (0.022),ireland (0.012),cdm (0.047)
3,want (0.010),produc (0.173),best (0.033),agreement (0.069),said (0.006),need (0.017),fyi (0.024),depart (0.041),said (0.005),pleas (0.027),url (0.019),republican (0.016),cameron (0.012),disapprov (0.041)
4,know (0.009),select (0.173),year (0.033),doc (0.069),one (0.005),support (0.013),memo (0.023),room (0.039),american (0.005),approv (0.026),keep (0.018),women (0.014),northern (0.011),jm (0.041)


In [67]:
TOP_K = 20

def parse_topic(column):
    parse = lambda row : (re.search(r'^([\w\s]+)\s\(', row).group(1),
                          re.search(r'\d+.\d+', row).group(0))
    parsed = map(parse, column)
    return list(parsed)

def extract_info(df_sentiment):
    out_dict = dict()
    for topic in df_sentiment:
        out_dict[topic] = parse_topic(df_sentiment[topic])
    return out_dict

parsed_positive = extract_info(pos_sentiment)
parsed_negative = extract_info(neg_sentiment)